In [1]:
import pandas as pd
import numpy as np
import urllib
import requests
from bs4 import BeautifulSoup
import os
from time import sleep
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError

In [2]:
df = pd.read_csv("links_correio_do_povo_organizado.csv")
df.shape


(189, 3)

In [3]:
df.head()

,ano,links,nome
0,2009,https://www.correiodopovo.com.br/not%C3%ADcias...,esquizofrenia
1,2010,https://www.correiodopovo.com.br/not%C3%ADcias...,esquizofrenia
2,2010,https://www.correiodopovo.com.br/not%C3%ADcias...,esquizofrenia
3,2010,https://www.correiodopovo.com.br/not%C3%ADcias...,esquizofrenia
4,2010,https://www.correiodopovo.com.br/not%C3%ADcias...,esquizofrenia


In [4]:
import spacy
nlp = spacy.load("pt_core_news_sm")

In [5]:
import time

In [6]:
def limpa_paragrafos(link, tamanho):
    
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36"}
    try:
        req = Request(link, headers = headers)
        response = urlopen(req)
       
    except HTTPError as e: # erro de segurança do site, pode ser necessário usar user-Agent
        print(e.status, e.reason)

    except URLError as e:
        print(e.reason)
    
#     html = urllib.request.urlopen(link).read()
    html = response.read()
    soup = BeautifulSoup(html)

    for script in soup(["script", "style"]):
        script.decompose()

    lista = []
    for i in soup.stripped_strings:
        lista.append(i)
    
#     print(lista)

    miolo = []
    for i in lista:
        if len(i) > tamanho:
            miolo.append(i.replace('\r\n',''))
#             print(miolo)

    paragrafos = []
    for i in range(len(miolo)):
        if 'esquizo' in miolo[i]:
            paragrafos.append(miolo[i])
    
    time.sleep(1)
    
#     for i in range(len(paragrafos)):
#         lista_final = []
#         doc = nlp(paragrafos[i].lower())
#         for token in doc:
#             valido = not token.is_stop and token.is_alpha
# #             valido = token.is_stop
#             if valido:
#                 lista_final.append(token.text)
    
#         limpo = ' '.join(lista_final)
#         paragrafos[i] = limpo
#         time.sleep(1)

    return paragrafos, miolo

In [7]:
link = df.links.values[15]
link

'https://www.correiodopovo.com.br/not%C3%ADcias/pol%C3%ADcia/m%C3%A3e-de-atirador-era-esquizofr%C3%AAnica-diz-secret%C3%A1rio-de-assist%C3%AAncia-social-1.59054'

In [8]:
paragrafos, miolo = limpa_paragrafos(link, 70)

In [9]:
str(paragrafos)

"['Mãe de atirador era esquizofrênica, diz secretário de Assistência Social', 'Mãe de atirador era esquizofrênica, diz secretário de Assistência Social', ', feriu pelo menos 13 e depois se suicidou, na manhã desta quinta-feira, na Escola Municipal Tasso da Silveira, em Realengo, zona Oeste do Rio de Janeiro - sofria de esquizofrenia, segundo o secretário estadual de Assistência Social e Direitos Humanos, Rodrigo Neves. No entanto, Wellington não apresentou qualquer sinal de desequilíbrio mental no período em que estudou na escola, entre 1999 e 2002.']"

In [10]:
str(miolo)

"['Mãe de atirador era esquizofrênica, diz secretário de Assistência Social', 'Mãe de atirador era esquizofrênica, diz secretário de Assistência Social', 'Rodrigo Neves afirma que Wellington Menezes de Oliveira não apresentava sinais de desequilíbrio mental', '07/04/2011 | 17:18\\n                    \\n          \\n          Atualizado\\n                    17:49', ', feriu pelo menos 13 e depois se suicidou, na manhã desta quinta-feira, na Escola Municipal Tasso da Silveira, em Realengo, zona Oeste do Rio de Janeiro - sofria de esquizofrenia, segundo o secretário estadual de Assistência Social e Direitos Humanos, Rodrigo Neves. No entanto, Wellington não apresentou qualquer sinal de desequilíbrio mental no período em que estudou na escola, entre 1999 e 2002.', '“O que espanta de fato é ele ter direcionado a sua ação para as crianças”, disse Neves, que esteve na escola. A Polícia Civil do Rio já sabe que Wellington vivia sozinho, nos últimos meses, em Sepetiba, na região metropolitana

In [11]:
df.links.iloc[1]

'https://www.correiodopovo.com.br/not%C3%ADcias/pol%C3%ADcia/cl%C3%ADnica-investigada-por-maus-tratos-%C3%A9-proibida-de-atender-adolescentes-1.46934'

In [12]:
n_links = df.shape[0]

In [14]:
for i in range(n_links):
    link = df.links.iloc[i]
    paragrafos, miolo = limpa_paragrafos(link, 70)
    df.loc[i, "paragrafos_resgatados"] = str(paragrafos)
    df.loc[i, "texto_completo"] = str(miolo)

In [15]:
t = df.query("paragrafos_resgatados == '[]'")
t

,ano,links,nome,paragrafos_resgatados,texto_completo
23,2012,https://www.correiodopovo.com.br/not%C3%ADcias...,esquizofrenia,[],['14/04/2012 | 16:46\n \n ...
35,2014,https://www.correiodopovo.com.br/not%C3%ADcias...,esquizofrenia,[],['01/04/2014 | 10:17\n \n ...
125,2022,https://www.correiodopovo.com.br/not%C3%ADcias...,esquizofrenia,[],['Comissão da Câmara aprova prisão a quem toca...
126,2022,https://www.correiodopovo.com.br/not%C3%ADcias...,esquizofrenia,[],['Justiça nega pedido de prisão de policiais e...
144,2013,https://www.correiodopovo.com.br/blogs/juremir...,esquizofrenico,[],"['Fico com a impressão de que quem acha ""norma..."
176,2020,https://www.correiodopovo.com.br/blogs/hiltorm...,esquizofrenica,[],"['""Continua o show do BBB. Uma casa de quarent..."
183,2010,https://www.correiodopovo.com.br/blogs/juremir...,esquizofrenicas,[],['Mas o Rio Grande do Sul não precisava ter re...


In [16]:
t.links.values 

array(['https://www.correiodopovo.com.br/not%C3%ADcias/pol%C3%ADcia/restos-mortais-de-v%C3%ADtimas-de-canibalismo-s%C3%A3o-enterrados-em-pe-1.89468',
       'https://www.correiodopovo.com.br/not%C3%ADcias/geral/anvisa-suspende-com%C3%A9rcio-de-lotes-de-rem%C3%A9dio-para-esquizofrenia-1.140120',
       'https://www.correiodopovo.com.br/not%C3%ADcias/pol%C3%ADtica/comiss%C3%A3o-da-c%C3%A2mara-aprova-pris%C3%A3o-a-quem-tocar-em-policial-durante-abordagem-1.839800',
       'https://www.correiodopovo.com.br/not%C3%ADcias/geral/justi%C3%A7a-nega-pedido-de-pris%C3%A3o-de-policiais-envolvidos-na-morte-de-genivaldo-1.839397',
       'https://www.correiodopovo.com.br/blogs/juremirmachado/dilma-furiosa-na-onu-e-psb-fervendo-no-rs-1.550',
       'https://www.correiodopovo.com.br/blogs/hiltormombach/eles-vivem-em-outro-planeta-1.409755',
       'https://www.correiodopovo.com.br/blogs/juremirmachado/votos-que-valem-mais-1.287780'],
      dtype=object)

In [17]:
# os links que não funcionaram são apenas TÍTULOS, ou trechos que contém menos de 120 caracteres

In [18]:
df["indice"] = [x for x in range(df.shape[0])]

In [19]:
df.to_csv("links_correio_do_povo_organizado_paragrafos_resgatados.csv", index = False)

In [20]:
t.to_csv("links_correio_do_povo_organizado_paragrafos_resgatados_sem_texto_descartar.csv", index = False)